In [236]:
import numpy as np
from tqdm import tqdm
import random

# Create Maze

Here I'm making a maze that uses just one digit ints so that it looks good and so that the most optimal path can be computed easily. A spoiler is that we will be using (9, 10) as the starting location and (0,5) as the ending location, noted by the 9. The optimal path is then:

[(9, 10), (9, 9), (9, 8), (9, 7), (8, 7), (7, 7), (7, 6), (7, 5), (6, 5), (5, 5), (5, 6), (5, 7), (4, 7), (3, 7), (2, 7), (1, 7), (1, 6), (1, 5), (0, 5)]

In [237]:
maze = [[-9,-9,-9,-9,-9, 9,-9,-9,-9,-9,-9],
        [-9,-1,-1,-1,-1,-1, 1,-1,-1,-1,-9],
        [-9,-1,-9,-9,-9,-9,-9,-1,-9,-1,-9],
        [-9,-1,-1,-1,-1,-1,-1,-1,-9,-1,-9],
        [-9,-9,-9,-1,-9,-9,-9,-1,-9,-9,-9],
        [-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1],
        [-9,-9,-9,-9,-9,-1,-9,-9,-9,-9,-9],
        [-9,-1,-1,-1,-1,-1,-1,-1,-1,-1,-9],
        [-9,-9,-9,-1,-9,-9,-9,-1,-9,-9,-9],
        [-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1],
        [-9,-9,-9,-9,-9,-9,-9,-9,-9,-9,-9]]

In [238]:
m_row = len(maze)
m_col = len(maze[0])

# Q learning

Because there are four options at every step of the maze, (up, down, left, right), we then create the necessary q table with the shape of (m_row, m_col 4) for the four directions

In [239]:
q_table = np.zeros((maze_row, maze_col,  4))

find the next action from the q_table to figure out which way to go

In [240]:
def get_next_action(row, col, epilson):
    if np.random.random() < epilson:
        return np.argmax(q_table[row,col])
    else:
        return np.random.randint(4)

Move in the direction of the directed action

In [241]:
def get_next_cell(row, col,step_index):
    up_right_down_left = [(row-1,col),(row,col+1),(row+1,col),(row,col-1)]
    x, y = up_right_down_left[step_index]
    if 0 <= x < maze_row and 0 <= y < maze_col:
        return up_right_down_left[step_index]
    else:
        return (row, col)

Here we have the two terminal states that we keep track of so that we know when we end our journey or if we come to a dead end.

In [242]:
terminal_state = [-9,9]

## Training

Get a random starting state that is not on the goal or on a wall. We use this random start for training purposes and such.

In [243]:
epilson = 0.9
discount_factor = 0.9
learning_rate = 0.9
epochs = 1000

def get_random_start():
    while True:
        row = np.random.randint(maze_row)
        col = np.random.randint(maze_col)
        if not maze[row][col] in terminal_state:
            return (row,col)

In [244]:
def train():
    for _ in tqdm(range(epochs)):
        row, col = get_random_start()

        while not maze[row][col] in terminal_state:
            action_index = get_next_action(row,col, epilson)

            old_row, old_col = row , col
            row , col = get_next_cell(row,col,action_index)

            reward = maze[row][col]
            old_q_val = q_table[old_row, old_col, action_index]
            temporal_diff = reward + (discount_factor * np.max(q_table[row, col])) - old_q_val

            new_q_val = old_q_val + (learning_rate * temporal_diff)
            q_table[old_row, old_col, action_index] = new_q_val

In [245]:
train()


100%|██████████| 1000/1000 [00:00<00:00, 7576.51it/s]

In [246]:
def get_shortest_path(row, col):
    if maze[row][col] in terminal_state:
        return []
    else: 
        cur_row , cur_col = row, col
        shortest_path = []
        shortest_path.append((cur_row, cur_col))
        while not maze[cur_row][cur_col] in terminal_state:
            action_index = get_next_action(cur_row, cur_col, 1.)
            cur_row, cur_col = get_next_cell(cur_row, cur_col, action_index)
            shortest_path.append((cur_row, cur_col))
        return shortest_path

In [247]:
human_pos = (9,10)
path = get_shortest_path(human_pos[0], human_pos[1])
print(path)

[(9, 10), (9, 9), (9, 8), (9, 7), (8, 7), (7, 7), (7, 6), (7, 5), (6, 5), (5, 5), (5, 6), (5, 7), (4, 7), (3, 7), (2, 7), (1, 7), (1, 6), (1, 5), (0, 5)]
